## Part 3 : Modeling_linear regression

In [3]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

In [2]:
df = pd.read_csv('/home/xc2418/final_proj/data/cleaned.csv')

/home/xc2418/ENTER/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,29,38,41,42,43,46,47,65,66,67,72,74) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [76]:
# linear regression

In [4]:
# data preprocessing
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics

/home/xc2418/ENTER/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
categorical = ['channelGrouping', 'device_browser', 'device_deviceCategory', 'device_operatingSystem',
               'geoNetwork_city', 'geoNetwork_country', 'geoNetwork_metro', 'geoNetwork_networkDomain',
               'geoNetwork_region', 'geoNetwork_subContinent', 'totals_bounces', 'totals_newVisits',
               'trafficSource_adwordsClickInfo.gclId', 'trafficSource_keyword', 'trafficSource_medium',
               'trafficSource_referralPath', 'trafficSource_source', 'hits_social.socialNetwork',
               'hits_transaction.currencyCode', 'customDimensions_index', 'customDimensions_value',
               'week', 'weekday', 'month']

numerical = ['totals_hits', 'totals_pageviews', 'visitNumber', 'visitStartTime', 'totals_sessionQualityDim',
             'totals_timeOnSite']

In [6]:
for col in categorical:
    lbe = preprocessing.LabelEncoder()
    lbe.fit(list(df[col].values.astype('str')))
    df[col] = lbe.transform(list(df[col].values.astype('str')))
    
for col in numerical:
    df[col] = df[col].astype(float)

In [7]:
# seperate the data into training and testing
train_df = df[df['year'] != 2018]
test_df = df[df['year'] == 2018]

In [8]:
train_x = train_df[numerical + categorical] 
test_x = test_df[numerical + categorical] 

In [9]:
train_y = train_df['y']
test_y = test_df['y']

In [10]:
# fit the linear model
lr = LinearRegression(normalize = True)
lr.fit(train_x, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [11]:
# coefficients
coef = pd.concat([pd.Series(numerical + categorical),pd.Series(lr.coef_)], axis = 1)

In [18]:
lr.intercept_

5.492112545288729

In [19]:
# predict y
y_pred = lr.predict(test_x)

In [20]:
# transform y to the original version
pred_trans = np.exp(y_pred) - 1
pred_trans[pred_trans < 0] = 0

In [21]:
compare_y = test_df[['fullVisitorId', 'totals_transactionRevenue']]

In [22]:
compare_y['pred_trans'] = pred_trans

/home/xc2418/ENTER/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
# calculate the loss as required
result = compare_y.groupby("fullVisitorId")["totals_transactionRevenue", "pred_trans"].sum()
result['y'] = np.log(result['totals_transactionRevenue'] + 1)
result['y_t'] = np.log(result['pred_trans'] + 1)
print(np.sqrt(metrics.mean_squared_error(result['y'],result['y_t'])))

1.5442804744652223
